## Извлечение ключевых слов

Или keyword extraction - это упрощенный вариант задачи саммаризации (иногда это называется реферирование), т.е. извлечения главной информации из текста. Настоящая саммаризация подразумевает, что главная информация описывается нормальными полными предложениями, но сделать это очень сложно. Извлечь только основные слова проще и задачу это решает тоже достаточно хорошо (по нескольким словам сразу приблизительно понятно, о чем текст и читать ключевые слова сильно быстрее, чем даже самое хорошее саммари). Еще одно преимущество ключевых слов - это то, что их удобно использовать в стандартном поиске (который работает только со словами и не анализирует последовательности). 

Одному тексту разные люди могут приписать разные ключевые слова, поэтому оценивать извлечение ключевых слов сложнее, чем обычную классификацию. Нужно либо для каждого текста иметь несколько наборов ключевых слов, либо брать тексты с ключевыми словами из разных источников. В любом случае с метрикам нужно быть аккуратными и смотреть больше на соотношение, чем на абсолютные значения. Ну и проверять полученные алгоритмы в реальной задаче или хотя бы на глаз. 

In [65]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [67]:
import numpy as np

In [66]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [10]:
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_0.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_1.jsonlines.zip

--2021-03-25 11:28:55--  https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_0.jsonlines.zip
Распознаётся github.com (github.com)… 140.82.121.3
Подключение к github.com (github.com)|140.82.121.3|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/mannefedov/ru_kw_eval_datasets/master/data/ng_0.jsonlines.zip [переход]
--2021-03-25 11:28:55--  https://raw.githubusercontent.com/mannefedov/ru_kw_eval_datasets/master/data/ng_0.jsonlines.zip
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 2987145 (2,8M) [application/zip]
Сохранение в: «ng_0.jsonlines.zip»

ng_0.jsonlines.zip  100%[===================>]   2,85M  4,40MB/s    за 0,6s    

2021-03-25 11:28:56 (4,40 MB/

In [11]:
!unzip ng_0.jsonlines.zip 
!unzip ng_1.jsonlines.zip

Archive:  ng_0.jsonlines.zip
  inflating: ng_0.jsonlines          
Archive:  ng_1.jsonlines.zip
  inflating: ng_1.jsonlines          


In [16]:

PATH_TO_DATA = './'

In [19]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA) if file.endswith('jsonlines')]

Объединим файлы в один датасет.

In [20]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:5], axis=0, ignore_index=True)

In [21]:
data.shape

(1987, 5)

In [22]:
data.head(3)

,keywords,title,url,content,summary
0,"[школа, образовательные стандарты, литература, история, фгос]","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html,"В среду состоялось отложенное заседание Совета по федеральным государственным образовательным стандартам (ФГОС) при Министерстве образования и науки РФ. Собрание должно было состояться еще в понедельник, но было перенесено по просьбе членов совета. И вот пришло сообщение, что общественники выразили согласие с позицией министерства. Новые ФГОСы приняты.\nНа вчерашнем заседании был принят ФГОС по начальной общеобразовательной школе. До 28 марта продлятся косультации по ФГОСам для средней школы.\nНапомним, что накануне Гильдия словесников разместила открытое письмо на имя министра образования и науки РФ Ольги Васильевой. По мнению авторов письма, новые ФГОСы грубо нарушают права детей, уже проучившихся по существующему стандарту до 6-го класса. Приняв новый стандарт, Министерство образования дает право контролирующим органам ловить детей на незнании большого списка произведений (235 за пять лет обучения). «Это исключает возможность полноценного их освоения, создает риск формального, п...","Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов виноваты издательства учебной литературы"
1,"[красота, законы]",У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрения. Плохо было б, если б она вовсе не озаряла своим светом космическую темень пустоты зрачка. Слава богу, такое вряд ли возможно. \nА случается, что красота уходит. Почему вдруг? И куда она девается, когда в один из философских обходов своего организма вы, еще недавно гордый ее обладатель, обескураженно ее недосчитываетесь? \nВообразите: прелестнейшее из созданий – ваша кошка пластичнейшими движениями рвет банкноту за банкнотой, забирается на карниз по шелковой занавеске или отгрызает полпаспорта. Где, скажите, теперь красота этой кошки? Или другой пример – с зазнобой сердца. Предмет романтичнейших грез наконец-то садится с вами на заветную скамейку в парке – закат, пение птах… И тут он силой своего обаяния с оглушительным плюхом обрушивает вокруг вас красоту и гармонию столетних дубов, тополей и прочего. Где, спрашивается, красота момента? \nЕсли от сказки после того, как ее рассказали,...",О живительной пользе укорота при выборе между плохим и хорошим
2,"[юзефович, гражданская война, пепеляев, якутия]",Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html,"Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» – она стала интеллектуальным бестселлером и классикой жанра – документальный роман. В то время автор попутно изучал и историю вооруженного восстания в Якутии в 1922–1923 годах под руководством Анатолия Пепеляева. И вот теперь из «якутского» материала сложилась отдельная книга. Тема ее для нынешнего читателя поистине раритетна. Ведь воевавший где-то на самом краю страны Пепеляев практически забыт, притом что о борьбе с ним когда-то в СССР выходили статьи и книги. В памяти потомков, образно говоря, от Пепеляева остался только пепел.\nЮзефович воскрешает в памяти не только его военные дела, но и человеческие черты. Этот провинциальный интеллигент, неврастеник и фаталист, начал восстание, практически не имея шансов на успех. Однако силою недюжинной харизмы Пепеляев сумел собрать вокруг себя многих боевых офицеров, таежных охотников и недовольных новыми порядками аборигенов. Для своих 32 лет ...",Крепость из тел и призрак независимой Якутии


In [68]:
data.shape

(1987, 8)

Каждой статье приписано какое-то количество ключевых слов. **Наша задача - придумать как извлекать точно такой же список автоматически.**
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [23]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [24]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем долго думать и тестрировать первое, что приходит в голову.

Возьмем первые 5 слов из заголовка.

In [25]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [26]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [70]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [71]:
evaluate(data['keywords'], data['title'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Теперь давайте посмотрим, что вообще извлекается.

In [29]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                      [ольга, васильева, обещала, "нг", не, перегружать, школьников]
1                                           [у, красоты, собственные, закон, и, воля]
2                                                            [апокалиптический, бунт]
3                   [f1., предсказать, результаты, гран-при, испании, было, несложно]
4                                               [возвращение, в, небесное, отечество]
5                                       [практическая, медицина, с, большим, будущим]
6                     [бумажный, носитель., вселенная,, человек,, язык, человеческий]
7                                      [в, багдаде, неспокойно,, а, будет, еще, хуже]
8    [ученые, предсказывают, появление, интуиции, у, искусственных, нейронных, сетей]
9                                  [россияне, от, вступления, в, вто, не, пострадают]
Name: title, dtype: object

In [30]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                            [и, в, –, по, на, что, с, о, к, не]
1                 [и, в, не, что, –, на, красота, вы, если, как]
2                     [в, и, не, на, –, от, о, из, пепеляев, но]
3                        [в, не, и, на, что, но, это, как, с, к]
4                          [и, в, –, не, он, с, на, что, а, его]
5                       [и, в, на, –, с, что, не, это, как, уже]
6                           [–, и, в, не, на, из, по, что, с, а]
7                       [в, и, не, на, но, ли, ираке, что, -, а]
8                           [и, в, на, о, с, –, как, а, что, ai]
9    [в, на, по, и, что, кристалина, –, россии, георгиева, том,]
Name: content, dtype: object

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [31]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [32]:
data['content_norm'] = data['content'].apply(normalize)

In [33]:
data['title_norm'] = data['title'].apply(normalize)

In [34]:
data['title_norm'].head(10)

0                           [ольга, васильев, обещать, нг, перегружать, школьник]
1                                             [красота, собственный, закон, воля]
2                                                        [апокалиптический, бунт]
3                       [f1, предсказать, результат, гран-при, испания, несложно]
4                                              [возвращение, небесный, отечество]
5                                      [практический, медицина, больший, будущий]
6                    [бумажный, носитель, вселенная, человек, язык, человеческий]
7                                                     [багдад, неспокойно, худой]
8    [учёный, предсказывать, появление, интуиция, искусственный, нейронный, сеть]
9                                        [россиянин, вступление, вто, пострадать]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [35]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [36]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


In [37]:
data['title_norm'].apply(lambda x: x[:10])

0           [ольга, васильев, обещать, нг, перегружать, школьник]
1                             [красота, собственный, закон, воля]
2                                        [апокалиптический, бунт]
3       [f1, предсказать, результат, гран-при, испания, несложно]
4                              [возвращение, небесный, отечество]
                                  ...                            
1982          [орбан, призвать, остановить, исламский, экспансия]
1983                  [парижский, соглашение, трудность, перевод]
1984                                     [троцкий, убить, кирпич]
1985                                [кино, город, сверкать, снег]
1986                       [информационный, война, цифровой, мир]
Name: title_norm, Length: 1987, dtype: object

Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [38]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [стандарт, который, источник, образовательный, фгоса, ольга, васильев, результат, предмет, исторический]
1                                          [красота, ваш, глаз, отчаяние, уйти, это, порыв, кошка, свой, один]
2                               [пепеляев, юзеф, книга, якутия, восстание, год, самый, война, место, когда-то]
3                              [гонка, команда, это, сказать, пилот, ferrari, mclaren, сезон, mercedes, время]
4                                          [есенин, поэт, клюев, год, свой, это, жизнь, который, смерть, 1925]
5                        [наш, медицина, медицинский, это, кафедра, выпускник, работать, уровень, работа, год]
6                               [книга, русский, человек, мозг, два, островной, говор, это, анатомия, который]
7                     [ирак, война, американец, это, партизанский, войско, структура, единый, сказать, важный]
8                       [который, нейросеть, свой, ai, клетка, искусственный, интеллект, самый, время, основа]
9

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [39]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [40]:
data['content_norm'] = data['content'].apply(normalize)

In [41]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.1


In [42]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)])

0               [стандарт, источник, фгоса, ольга, васильев, результат, предмет, школа, письмо, произведение]
1                                 [красота, глаз, отчаяние, порыв, кошка, предмет, дело, руина, место, мечта]
2                               [пепеляев, юзеф, книга, якутия, восстание, год, война, место, леонид, сибирь]
3                               [гонка, команда, пилот, сезон, время, машина, место, круг, гран-при, испания]
4                               [есенин, поэт, клюев, год, жизнь, смерть, сергей, человек, борода, мариенгоф]
                                                        ...                                                  
1982                       [ес, страна, политика, европа, брюссель, венгрия, правый, орбан, выборы, евросоюз]
1983                         [выброс, развитие, цель, газ, уровень, год, соглашение, сторона, климат, страна]
1984                          [коваль, юрий, писатель, книга, человек, слово, клуб, иосич, осина, милиционер]
1985      

Ещу улучшения!

In [43]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0    [стандарт, источник, фгоса, ольга, васильев, результат, предмет, школа, письмо, произведение]
1                      [красота, глаз, отчаяние, порыв, кошка, предмет, дело, руина, место, мечта]
2                    [пепеляев, юзеф, книга, якутия, восстание, год, война, место, леонид, сибирь]
3                    [гонка, команда, пилот, сезон, время, машина, место, круг, гран-при, испания]
4                    [есенин, поэт, клюев, год, жизнь, смерть, сергей, человек, борода, мариенгоф]
5       [медицина, кафедра, выпускник, уровень, работа, год, практика, ординатор, рудна, обучение]
6               [книга, человек, мозг, говор, анатомия, глава, звезда, вопрос, слово, азербайджан]
7       [ирак, война, американец, войско, структура, принцип, взрыв, центр, террорист, количество]
8    [нейросеть, клетка, интеллект, время, основа, изображение, задача, работа, система, внимание]
9      [россия, вступление, кристалина, георгиев, вто, банка, страна, переговоры, директор, слово]
Name: cont

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [44]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [80]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)

In [81]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [82]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [83]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [84]:
## так как матрица в tfidf в спарс формате,  ее нельзя просто так отсортировать
## перевести ее в обычный формат для всех данных тоже не получится - не хватит памяти
## поэтому пройдем по строчкам, переведем строчку в обычный array и отсортируем ее
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [85]:
keywords[:3]

[['фгоса',
  'ольга васильев',
  'стандарт',
  'васильев',
  'ольга',
  'источник',
  'предмет',
  'событие явление',
  'произведение',
  'письмо'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'руина',
  'порыв отчаяние',
  'красота глаз',
  'ваби',
  'вопрос красота'],
 ['пепеляев',
  'юзеф',
  'якутия',
  'восстание',
  'книга',
  'восстание якутия',
  'пепеляев боец',
  'анатолий пепеляев',
  'начало 20',
  'леонид юзеф']]

In [86]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за **baseline.**

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [52]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [92]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n
    


In [94]:
from tqdm import tqdm
tqdm.pandas()

In [95]:
%%time
keywords_rw = data['content_norm'].progress_apply(lambda x: get_kws(x, 10, 10))

100%|██████████| 1987/1987 [01:40<00:00, 19.72it/s]

CPU times: user 1min 38s, sys: 8.43 s, total: 1min 47s
Wall time: 1min 40s


In [55]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [56]:
keywords_rw.head(10)

0       [событие, письмо, стандарт, фгоса, школа, источник, васильев, процесс, произведение, ольга]
1                [красота, глаз, кошка, отчаяние, место, порыв, занавеска, вопрос, дерево, предмет]
2                [пепеляев, якутия, книга, юзеф, собрание, поэт, красных, начало, 20-ха, восстание]
3                   [команда, гонка, пилот, круг, машина, борьба, сезон, время, испания, себастьян]
4                          [есенин, год, поэт, жизнь, клюев, борода, гроб, будущее, тишь, исповедь]
5         [кафедра, медицина, уровень, выпускник, гражданин, год, вопрос, качество, обучение, врач]
6                     [глава, говор, вопрос, небо, человек, звезда, верея, исследование, мозг, мир]
7          [ирак, война, штаб, узел, американец, снабжение, структура, хусейн, специалист, бригада]
8    [нейросеть, клетка, время, задача, основа, изображение, интеллект, вычисление, фото, внимание]
9       [переговоры, директор, вто, вступление, георгиев, россия, слово, кристалина, страна, линия]


Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [57]:
import networkx as nx

In [89]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    # менять тут 
    node2measure = dict(nx.pagerank(G)) 
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

Тут использован PageRank как метрика центральности. Про другие можно узнать вот тут - https://networkx.github.io/documentation/stable/reference/algorithms/centrality.html

Попробуйте разные метрики. Некоторые могут работать достаточно долго

In [90]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 3min 29s, sys: 1.37 s, total: 3min 30s
Wall time: 3min 32s


In [91]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.12
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

Готовое решение есть в gensim. Давайте попробуем его.

In [62]:
from gensim.summarization import keywords

In [63]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])

In [64]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


Наша имплементация отработала получше.

## Домашнее задание

В семинаре использовался только небольшой кусочек данных. На всех данных пересчитайте baseline (tfidf). 

**Ваша задача - предложить 3 способа побить бейзлайн на всех данных.**

Нет никаких ограничений кроме:

1) нельзя изменять метрику  
2) решение должно быть воспроизводимым  
3) способы дожны отличаться друг от друга не только гиперпараметрами (например, нельзя три раза поменять гиперпарамтры в TfidfVectorizer и сдать работу)  
4) изменение количества извлекаемых слов не является улучшением (выберите одно значение и используйте только его)  

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - 

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

В поисках идей можно почитать обзоры по теме (посмотрите еще статьи, в которых цитируются эти обзоры): https://www.semanticscholar.org/search?year%5B0%5D=2012&year%5B1%5D=2020&publicationType%5B0%5D=Reviews&q=keyword%20extraction&sort=relevance

**Использовать доступные готовые решения тоже можно**. Так что погуглите перед тем, как приступать к заданию. 